Bibliothèques utiles :

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from cfd_common_article import *
from math import *
import time
import scipy

In [2]:
%matplotlib qt

In [3]:
plt.ion()

Choix du domaine et des paramètres de la discrétisation du problème :

In [4]:
##### Domain #####

# Domain size
Lx = 2.
Ly = 2.

# Physical grid size
Nx_phys = 100
Ny_phys = 100
Δx = Lx/Nx_phys
Δy = Ly/Ny_phys

# Numerical grid size, including ghost points
Nx = Nx_phys + 2
Ny = Ny_phys + 2

# Physical grid (x,y)
# Due to the "staggered-like" approach to boundary conditions with ghost points,
#  the border go through the middle of border cells
x = np.linspace(Δx/2, Lx-Δx/2, Nx_phys) 
y = np.linspace(Δy/2, Ly-Δy/2, Ny_phys)
[xx,yy] = np.meshgrid(x,y) 

Initialisation des champs de vitesse / champ de pression et champ de densité :

On note $u,v$ les composantes horizontale, verticales du champ de vitesse. On les initialise à :...

In [5]:
##### Fields #####

# Velocity field (u,v)[i,j], initialized to "zero"
u = np.zeros((Nx,Ny))
v = np.zeros((Nx,Ny))

vrand_ampli = 0.5
#u += vrand_ampli * ( 1-2*np.random.rand(Nx,Ny) ) # random vertical velocity to speed up the instability
#v[1:-1,1:-1] = 1/2 * np.where(((xx-0*Lx/2)**2+(yy-Ly/2)**2)<0.5, 1, 0)
#v[1:-1,1:-1] = 1/2 * np.where(((xx-0*Lx/2)**2+(yy-Ly/2)**2)<0.3, v[1:-1,1:-1], 0)
#v[1:-1,1:-1] = 1/2 * np.transpose(np.where(((xx-Lx/2)**2+(yy-0.15*Ly)**2)<0.2**2, 1, 0))
# Pressure field press[i,j] and its gradient (dx_press,dy_press)
press = np.zeros((Nx,Ny))
dx_press = np.zeros((Nx,Ny))
dy_press = np.zeros((Nx,Ny))

# Density field
ρ0 = 5
ρ = ρ0*np.ones((Nx,Ny))
#ρ[:, 4*Ny//10 : 8*Ny//10] = 1
ρ[1:-1,1:-1] = np.where(((xx-0*Lx/2)**2+(yy-Ly/2)**2)<0.5, 5, 1)
ρ[1:-1,1:-1] = np.where(((xx-0*Lx/2)**2+(yy-Ly/2)**2)<0.3, 1, ρ[1:-1,1:-1])
ρ[1:-1,1:-1] = np.transpose(np.where(((xx-Lx/2)**2+(yy-0.15*Ly)**2)<0.2**2, 1, 5))

On impose les conditions aux bords (sur les GhostPoints) :

In [6]:
def VelocityGhostPointsFlatten(u,v):
    ### left
    u[0:Nx],v[0:Nx] = 0,0
    #u[0:Nx],v[0:Nx] = u[Nx:2*Nx],v[Nx:2*Nx]
    ### right
    u[-Nx:],v[-Nx:] = 0,0
    #u[-Nx:],v[-Nx:] = u[-2*Nx:-Nx],v[-2*Nx:-Nx]
    ### bottom
    u[0::Ny],v[0::Ny] = 0,0
    #u[0::Ny],v[0::Ny] = u[1::Ny],v[0::Ny]
    ### top          
    u[Nx-1::Ny],v[Nx-1::Ny] = 0,0
    #u[Nx-1::Ny],v[Nx-1::Ny] = u[Nx-2::Ny],v[Nx-1::Ny]
def VelocityGhostPointsFlatten(u,v):
    ### left
    u[0:Nx],v[0:Nx] = 0,0
    #u[0:Nx],v[0:Nx] = u[Nx:2*Nx],v[Nx:2*Nx]
    ### right
    u[-Nx:],v[-Nx:] = 0,0
    #u[-Nx:],v[-Nx:] = u[-2*Nx:-Nx],v[-2*Nx:-Nx]
    ### bottom
    u[0::Ny],v[0::Ny] = 0,0
    #u[0::Ny],v[0::Ny] = u[1::Ny],v[0::Ny]
    ### top          
    u[Nx-1::Ny],v[Nx-1::Ny] = 0,0
    #u[Nx-1::Ny],v[Nx-1::Ny] = u[Nx-2::Ny],v[Nx-1::Ny]
def PhiGhostPoints(phi):
    ### left
    phi[0:Nx] = phi[Nx:2*Nx]
    ### right
    phi[-Nx:] = phi[-2*Nx:-Nx]
    ### bottom               
    phi[0::Ny] = phi[1::Ny]
    ### top            
    phi[Nx-1::Ny] = phi[Nx-2::Ny]
def get_density_advected(ρ,u,v):
    u = u.reshape((Nx,Ny), order = 'F')
    v = v.reshape((Nx,Ny), order = 'F')
    ρ = ρ.reshape((Nx,Ny), order = 'F')
    # Left:
    ρ_l = np.where(u[1,:]<0,Δt/Δx*(u[1,:]*ρ[1,:]-u[0,:]*ρ[0,:])+ ρ[0,:],Δt/Δx*u[0,:]*(1*-ρ[0,:])+ρ[0,:])
    ρ[0,:] = 5
    # Right:
    ρ_r = np.where(u[-2,:]>0,Δt/Δx*(u[-2,:]*ρ[-2,:]-u[-1,:]*ρ[-1,:])+ρ[-1,:],Δt/Δx*u[-1,:]*(1*-ρ[-1,:])+ρ[-1,:])
    ρ[-1,:] = 5
    # Botton:
    ρ_b = np.where(v[:,1]<0,Δt/Δy*(u[:,1]*ρ[:,1]-u[:,0]*ρ[:,0])+ρ[:,0],Δt/Δy*u[:,0]*(1*-ρ[:,0])+ρ[:,0])
    ρ[:,  0] = 5
    # Top:
    ρ_t = np.where(v[:,-2]<0,Δt/Δy*(u[:,-2]*ρ[:,-2]-u[:,-1]*ρ[:,-1])+ρ[:,-1],Δt/Δy*u[:,-1]*(1*-ρ[:,-1])+ρ[:,-1])
    ρ[:, -1] = 5
    ρ = ρ.flatten('F')
    u = u.flatten('F')
    v = v.flatten('F')  

In [7]:
def VelocityGhostPoints(u,v):
    u,v = u.reshape((Nx,Ny), order = 'F'),v.reshape((Nx,Ny), order = 'F')
    # ya pas cette histoire de "frontière = au milieu des cellules de bord ?"
    ### left
    u[0, :] = 0*u[1, :]
    v[0, :] = 0*v[1, :]
    ### right    
    u[-1, :] = 0*u[-2, :]    # outflow condition => dérivée nulle
    v[-1, :] = 0*v[-2, :]    # outflow condition => dérivée nulle
    ### bottom
    u[:,  0] = 0  # no slip
    v[:,  0] = 0  # imperméabilité
    ### top     
    u[:, -1] = 0*u[:, -2]
    v[:, -1] = 0*v[:, -2]
    u,v = u.flatten('F'),v.flatten('F')
    
def PhiGhostPoints(phi):
    phi = phi.reshape((Nx,Ny), order = 'F')
    ### left
    phi[0, :] = phi[1, :]
    ### right
    phi[-1, :] = phi[-2, :]
    ### bottom               
    phi[:,  0] = phi[:,  1]
    ### top            
    phi[:, -1] = phi[:, -2]
    phi = phi.flatten('F')
def get_density_advected(ρ,u,v):
    u = u.reshape((Nx,Ny), order = 'F')
    v = v.reshape((Nx,Ny), order = 'F')
    ρ = ρ.reshape((Nx,Ny), order = 'F')
    # Left:
    ρ[0,:] = 5
    # Right:
    ρ[-1,:] = 5
    # Botton:
    ρ[:,  0] = 5
    # Top:
    ρ[:, -1] = 5
    ρ = ρ.flatten('F')
    u = u.flatten('F')
    v = v.flatten('F')  

Ensuite vient la définition de notre schéma :

In [8]:
def plot_density(ρ):
    plt.figure()
    ρ_im = ρ.reshape((Nx,Ny),order = 'F')
    plt.imshow(np.transpose(ρ_im), origin='lower', extent=(0,Lx,0,Ly), cmap=plt.cm.RdPu)
    plt.colorbar()
    plt.show()
def plot_scal_and_vel(ρ,u,v):
    plt.figure()
    modx = Nx//50
    mody = Ny//50
    ρ_im = ρ.reshape((Nx,Ny), order = 'F')[1:-1,1:-1]
    ur, vr = u.reshape((Nx,Ny), order = 'F'), v.reshape((Nx,Ny), order = 'F')
    plt.imshow(np.transpose(ρ_im), origin='lower', extent=(0,Lx,0,Ly), cmap=plt.cm.RdPu)
    plt.quiver(xx[::modx,::mody], yy[::modx,::mody], np.transpose(ur[1:-1:modx,1:-1:mody]), np.transpose(vr[1:-1:modx,1:-1:mody]), scale=40, angles = 'uv')
    plt.colorbar()
    plt.show()
def plot_mat(Mat, Title = None):
    Mat2 = Mat.todense()
    plt.figure()
    plt.imshow(Mat2)
    if Title:
        plt.title(Title)
    plt.colorbar()
    plt.show()
def plot_vect(Vect, Title = None):
    Vect2 = Vect.reshape((Nx,Ny))
    plt.imshow(Vect2.transpose())
    if Title:
        plt.title(Title)
    plt.colorbar()
    plt.show()

In [9]:
###### CONSTRUCTION des matrices et LU decomposition

### Matrix construction for projection step
LAP = FD_2D_Laplacian_matrix (Nx, Ny, Δx, Δy, BCdir_left=True, BCdir_right=True, BCdir_top=True, BCdir_bot=True) 
LAP[1,2] = 1/(Δx*Δy)


In [10]:
#p = np.ones((Nx*Ny))
#Mat = LAP + Deltax + Deltay
#phi = lg.spsolve(Mat, p)
#B = Mat@phi
#plot_density(B)

In [11]:
def Divergence_flat (u,v, Δx,Δy):
    div = Deltax@u + Deltay@v
    return div

In [12]:
### Méthode de l'article :
## Matrices utiles:
datax = [-np.ones(Nx-1), np.zeros(Nx), np.ones(Nx-1)]
datax[1][0] = -2
datax[2][0] = 2
datax[0][-1] = -2
datax[1][-1] = 2
datay = [-np.ones(Ny-1), np.zeros(Ny), np.ones(Ny-1)]
datay[1][0] = -2
datay[2][0] = 2
datay[0][-1] = -2
datay[1][-1] = 2
offsets = np.array([-1,0,1])
dx,dy = 1/Δx/2, 1/Δy/2
DX = sp.diags(datax,offsets, shape=(Nx,Nx),format = 'csr') * dx
DY = sp.diags(datay,offsets, shape=(Ny,Ny),format = 'csr') * dy
Deltax = sp.kron(sp.eye(Ny,Ny), DX,format = 'csr')
Deltay = sp.kron(DY, sp.eye(Nx,Nx), format = 'csr')
Id = sp.diags(np.ones((Nx*Ny)), 0, shape = (Nx*Ny,Nx*Ny), format = 'csr')
zero = np.zeros((Nx*Ny))
A = np.ones((Nx*Ny))
α = 0*pi/4
fx,fy = +sin(α) * A, -cos(α) *  A #force extérieure
    
def grad(s,Δx,Δy,Nx,Ny):
    return Deltax@s,Deltay@s

def advect(u,v,Δx,Δy,Nx,Ny):
    uf = sp.diags(u.flatten('F'),0,format = 'csr')
    vf = sp.diags(v.flatten('F'),0,format = 'csr')
    return uf@Deltax + vf@Deltay

def get_rho(u,v,ρ,Nx,Ny,Δx,Δy,Δt): #Corr1
    A = Id + Δt*advect(u,v,Δx,Δy,Nx,Ny)
    #plot_mat(A)
    return lg.spsolve(A,ρ)

def approx_u(u,v,ρ,ρ_,s,fx,fy,Δx,Δy,Δt,LAP,Nx,Ny): #prend des entrées flattens pour la densité
    a,b = sp.diags(ρ,0,shape = (Nx*Ny,Nx*Ny), format = 'csr'),sp.diags(1/ρ_,0,shape = (Nx*Ny,Nx*Ny), format = 'csr')
    A = Id + Δt* b@a@advect(u,v,Δx,Δy,Nx,Ny) - Δt/Re * b@LAP
    Nabsx,Nabsy = grad(s,Δx,Δy,Nx,Ny)
    RHSx = b@(Δt*fx-Δt/Re * Nabsx) + u.flatten('F')
    RHSy = b@(Δt*fy-Δt/Re * Nabsy) + v.flatten('F')
    u_hatx = lg.spsolve(A,RHSx)
    u_haty = lg.spsolve(A,RHSy)
    return u_hatx,u_haty

def get_phi(ρ,u_hatx,u_haty,Δx,Δy,Nx,Ny,LAP):
    ρ_inv = sp.diags(1/ρ,0, shape = (Nx*Ny,Nx*Ny),format = 'csr')
    LapPart = ρ_inv @ LAP
    Dx_ρ, Dy_ρ = grad(1/ρ,Δx,Δy,Nx,Ny)
    Dx_ρ, Dy_ρ = sp.diags(Dx_ρ,0,shape = (Nx*Ny,Nx*Ny), format = 'csr'),sp.diags(Dy_ρ,0,shape = (Nx*Ny,Nx*Ny), format = 'csr')
    Mat = LapPart + Dx_ρ@Deltax+ Dy_ρ@Deltay
    RHS = -Divergence_flat(u_hatx,u_haty, Δx,Δy)
    phi = lg.spsolve(Mat, RHS)
    return phi

def update_u_s(s,ρ,phi,u_hatx,u_haty,Δx,Δy):
    dX,dY = grad(phi,Δx,Δy,Nx,Ny)
    ui = u_hatx + 1/ρ * dX
    vi = u_haty + 1/ρ * dY
    si = s - Divergence_flat(u_hatx,u_haty, Δx,Δy)
    return ui,vi,si

In [13]:
## Check mass
def mass_check(rho):
    return Δx*Δy*np.sum(rho)

mass_check(ρ)

20.302400000000002

Paramètres de la boucle :

In [14]:
### ATTENTION: dt_init calculer la CFL a chaque iteration... 
Δt = 0.01

t = 0. # total time

# parameters
Re = 10000 # Reynolds number
g = 1
s = np.zeros((Nx*Ny),dtype = 'float64') #flatten

#fig = plt.figure(1, figsize=(14,7))

loop_continue = True
def on_close (event):
    global loop_continue
    loop_continue = False
#fig.canvas.mpl_connect('close_event', on_close)


niter = 0
disp_modulo = 1

In [18]:
import warnings
#warnings.filterwarnings('error')
# Pour des raisons pratiques, toutes les variables sont constamment flatten.
# On ne reconstruit que pour l'affichage
u = u.flatten('F')
v = v.flatten('F')
s = np.zeros((Nx*Ny),dtype = 'float64') #flatten

niter = 0
ims = [ρ]
imu = [u]
imv = [v]
while niter<11:
    # Step 1: A PRIORI C'EST JUSTE
    ρ_ = ρ.flatten('F').copy()
    ρ = get_rho(u,v,ρ_,Nx,Ny,Δx,Δy,Δt)
    ims.append(ρ)
    get_density_advected(ρ,u,v)
    # Step 2: 
    u_hatx, u_haty = approx_u(u,v,ρ,ρ_,s,fx,fy,Δx,Δy,Δt,LAP,Nx,Ny)
    VelocityGhostPoints(u_hatx,u_haty)
    #ims.append(u_hatx)
    #ims.append(u_hatx)
    # Step 3:
    phi = get_phi(ρ,u_hatx,u_haty,Δx,Δy,Nx,Ny,LAP)
    plot_density(phi)
    #ims.append(phi)
    #PhiGhostPoints(phi)
    # Step 4:
    u,v,s = update_u_s(s,ρ,phi,u_hatx,u_haty,Δx,Δy)
    #ims.append(u)
    #ims.append(v)
    imu.append(u)
    imv.append(v)
    #if niter % 10 ==0:
        #plot_density(phi)
        #plot_scal_and_vel(ρ,u,v)
    t += Δt
    niter += 1

In [19]:
for i in range(11):
    plot_scal_and_vel(ims[i],imu[i],imv[i])

In [17]:
inv = sp.diags(1/ρ,0,shape = ((Nx*Ny),(Nx*Ny)))